In [2]:
import ast
import numpy as np
import pandas as pd

df = pd.read_csv("train.csv")
df

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

input_texts = df['premise'].tolist()
device = "cuda:1" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large").to(device)

# Set the batch size for inference
batch_size = 4  # Set your desired batch size

# Calculate the total number of batches
total_batches = len(input_texts) // batch_size
if len(input_texts) % batch_size != 0:
    total_batches += 1

all_embeddings = []

# Process in batches
for i in range(total_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, len(input_texts))
    batch_texts = input_texts[start_index:end_index]

    # Tokenize the input texts for this batch
    batch_dict = tokenizer(batch_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)

    # Perform inference on this batch
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = outputs.last_hidden_state[:, 0]

    all_embeddings.append(embeddings)

    # Clear GPU memory
    del batch_dict
    del outputs
    torch.cuda.empty_cache()

# Concatenate embeddings from all batches
premise_embeddings = torch.cat(all_embeddings, dim=0)

print(len(premise_embeddings))

/home/ttsai/miniconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


12120


In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

input_texts = df['hypothesis'].tolist()
device = "cuda:1" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large").to(device)

# Set the batch size for inference
batch_size = 4  # Set your desired batch size

# Calculate the total number of batches
total_batches = len(input_texts) // batch_size
if len(input_texts) % batch_size != 0:
    total_batches += 1

all_embeddings = []

# Process in batches
for i in range(total_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, len(input_texts))
    batch_texts = input_texts[start_index:end_index]

    # Tokenize the input texts for this batch
    batch_dict = tokenizer(batch_texts, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)

    # Perform inference on this batch
    with torch.no_grad():
        outputs = model(**batch_dict)
        embeddings = outputs.last_hidden_state[:, 0]

    all_embeddings.append(embeddings)

    # Clear GPU memory
    del batch_dict
    del outputs
    torch.cuda.empty_cache()

# Concatenate embeddings from all batches
hypothesis_embeddings = torch.cat(all_embeddings, dim=0)

print(len(hypothesis_embeddings))

12120


In [5]:
df1 = pd.DataFrame(premise_embeddings.cpu())
df2 = pd.DataFrame(hypothesis_embeddings.cpu())

new_columns = [i for i in range(1024, 2049)]

df2 = df2.rename(columns=dict(zip(df2.columns, new_columns)))
combined_df = pd.concat([df1, df2], axis=1)
combined_df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.773477,0.172458,-0.314092,-0.678002,1.384502,-1.248385,-0.293865,3.040153,2.263361,-0.778844,...,-1.030369,-0.906297,1.242696,-0.501899,-0.211161,1.191084,0.469429,0.110842,-0.149772,-0.497522
1,0.657311,-0.017385,-1.118371,-1.304048,0.766887,-0.803735,-0.551836,2.328789,1.700089,-0.618788,...,-0.719334,-0.542218,1.108367,-0.678961,-0.497115,0.578400,0.584015,-0.216586,0.283829,0.375096
2,0.547934,0.410443,-0.901785,-1.223757,0.732121,-1.177999,0.261307,1.797085,1.759348,-0.419995,...,-1.121317,-1.544234,1.999434,-0.040087,0.292896,1.352047,0.491452,-0.665395,-0.242141,0.525729
3,1.283637,0.507273,-1.172738,-0.902861,0.802482,-1.168283,-0.337774,2.087637,1.823133,-0.605531,...,-0.152426,-0.893181,1.614251,-0.466055,-0.343896,0.422310,1.219885,0.270496,-0.409826,0.042990
4,0.954977,-0.400774,-0.918558,-0.922931,0.551224,-0.366295,-0.069381,1.310568,1.284144,-1.602545,...,0.060879,-0.606343,1.515249,-0.399388,-0.316173,1.828813,-0.458810,-0.206368,-0.837639,-0.790726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12115,0.742992,-0.316537,-1.942790,-1.271411,1.504665,-0.375486,-0.616113,2.001062,2.240077,-1.333948,...,-0.517813,-0.380195,1.220592,0.076595,-0.154977,0.795626,0.391578,-0.029183,-0.686130,0.868474
12116,0.726459,-0.541171,-1.116222,-0.964100,0.148217,-0.677949,-0.331835,2.542156,1.322369,-0.471769,...,-0.922750,-1.115611,1.370776,-0.465410,-0.499819,0.537749,0.275970,-0.314839,-1.190199,-0.465968
12117,0.816322,0.482108,-0.817469,-0.588759,0.410960,-0.632837,-0.771746,2.160142,1.873152,-0.224282,...,-0.990657,-0.313674,1.357502,-1.188595,-0.081460,0.750679,0.622086,0.475333,-0.796993,0.251659
12118,1.414680,-0.618776,-0.769140,-1.568142,1.082300,-1.498607,-0.218620,1.906582,0.791539,-2.163370,...,-0.407540,-1.152847,1.299198,-0.440505,0.219434,-0.160850,-0.460752,0.237107,-1.148381,0.386109


In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(combined_df, df['label'], test_size=0.2, random_state=42)

model1 = xgb.XGBClassifier(objective='multi:softprob',gpu_id=1,booster='gbtree',n_estimators=500, eval_metric='merror',max_depth=6)  # num_classes 是类别的数量

model1.fit(X_train, y_train,eval_set=[(X_test, y_test)],verbose=True)

y_pred = model1.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))


/home/ttsai/miniconda3/envs/llm/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [21:56:23] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)


[0]	validation_0-merror:0.56353
[1]	validation_0-merror:0.53837
[2]	validation_0-merror:0.52269
[3]	validation_0-merror:0.52310
[4]	validation_0-merror:0.51279
[5]	validation_0-merror:0.50660
[6]	validation_0-merror:0.50289
[7]	validation_0-merror:0.49835
[8]	validation_0-merror:0.48969
[9]	validation_0-merror:0.48845
[10]	validation_0-merror:0.48762
[11]	validation_0-merror:0.48927
[12]	validation_0-merror:0.48309
[13]	validation_0-merror:0.48639
[14]	validation_0-merror:0.48474
[15]	validation_0-merror:0.48556
[16]	validation_0-merror:0.48226
[17]	validation_0-merror:0.48267
[18]	validation_0-merror:0.49051
[19]	validation_0-merror:0.48886
[20]	validation_0-merror:0.48515
[21]	validation_0-merror:0.48804
[22]	validation_0-merror:0.48804
[23]	validation_0-merror:0.48680
[24]	validation_0-merror:0.48639
[25]	validation_0-merror:0.48845
[26]	validation_0-merror:0.48556
[27]	validation_0-merror:0.48762
[28]	validation_0-merror:0.48969
[29]	validation_0-merror:0.48639
[30]	validation_0-me

/home/ttsai/miniconda3/envs/llm/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [21:57:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:1, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
